In [ ]:
# pip install spacy==2.3.5
# python -m spacy download en
# pip install neuralcoref --no-binary neuralcoref (got binary error on github so this install)
# !pip install spacy-experimental

# neuralcoref==4.0 - not used anymore 
# spacy==2.3.5 - old version to support neuralcoref


# Python==3.8 (3.7.10 backup plan)

#!pip install https://github.com/explosion/spacy-experimental/releases/download/v0.6.1/en_coreference_web_trf-3.4.0a2-py3-none-any.whl
!pip install transformers
!pip install fastcoref


In [ ]:

# Load your usual SpaCy model (one of SpaCy English models)
# import spacy
# nlp = spacy.load('en')

# Add neural coref to SpaCy's pipe
# import neuralcoref
# neuralcoref.add_to_pipe(nlp)
# 
# # You're done. You can now use NeuralCoref as you usually manipulate a SpaCy document annotations.
# doc = nlp(u'My sister has a dog. She loves him.')
# 
# doc._.has_coref
# doc._.coref_clusters

In [ ]:
# # Access the resolved coreferences
# for token in doc:
#     print(token.text, token._.in_coref, token._.coref_clusters)
# 
# # Iterate over resolved coreference clusters
# for cluster in doc._.coref_clusters:
#     print(cluster.mentions)

In [ ]:
import trafilatura
downloaded = trafilatura.fetch_url('https://github.blog/2019-03-29-leader-spotlight-erin-spiceland/')
trafilatura.extract(downloaded)

In [ ]:
from trafilatura import feeds
# mylist = feeds.find_feed_urls('https://www.dailymail.co.uk/news/uk-politics/index.rss')
mylist = feeds.find_feed_urls('https://www.dailymail.co.uk/news/rishi-sunak/index.rss')
print(mylist)

https://adrien.barbaresi.eu/blog/how-to-download-parallel-politeness-rules-python.html
Provides a method to check the robot txt of a site and check if the url being extracted meets the politeness policy.

When extracting from bing check for each domain and store in data structure with allow/disallow. If the same base url appears again then no need to recheck robot.txt

In [38]:
import urllib.robotparser

# define a website to look for rules
base_url = "https://www.theguardian.com"

# load the necessary components, fetch and parse the file
rules = urllib.robotparser.RobotFileParser()
rules.set_url(base_url + "/robots.txt")
rules.read()

print(rules)

# determine if a page can be fetched by all crawlers
rules.can_fetch("*", "https://www.theguardian.com/import urllib.robotparser")
# returns True or False

User-agent: Mediapartners-Google
Allow: 

User-agent: NewsNow
Disallow: /

User-agent: GPTBot
Disallow: /

User-agent: CCBot
Disallow: /

User-agent: TurnitinBot
Disallow: /

User-agent: PetalBot
Disallow: /

User-agent: MoodleBot
Disallow: /

User-agent: *
Disallow: /sendarticle/
Disallow: /Users/
Disallow: /users/
Disallow: /%2A/print%24
Disallow: /email/
Disallow: /contactus/
Disallow: /share/
Disallow: /websearch
Disallow: /%2A%3Fcommentpage%3D
Disallow: /whsmiths/
Disallow: /external/overture/
Disallow: /discussion/report-abuse/%2A
Disallow: /discussion/report-abuse-ajax/%2A
Disallow: /discussion/comment-permalink/%2A
Disallow: /discussion/report-abuse/%2A
Disallow: /discussion/user-report-abuse/%2A
Disallow: /discussion/handlers/%2A
Disallow: /discussion/your-profile
Disallow: /discussion/your-comments
Disallow: /discussion/edit-profile
Disallow: /discussion/search/comments
Disallow: /discussion/%2A
Disallow: /search
Disallow: /music/artist/%2A
Disallow: /music/album/%2A
Disallow

True

In [ ]:
downloaded = trafilatura.fetch_url(mylist[6])
trafilatura.extract(downloaded, favour_precision=True, include_comments=False, include_images=False, include_tables=False)

In [ ]:
downloaded = trafilatura.fetch_url('https://www.dailymail.co.uk/news/article-12350195/Think-twice-imposing-Ulez-tax-hard-working-Brits-Rishi-Sunak-blasts-Sadiq-Khan-hated-low-emission-zone-expansion-plan.html?ns_mchannel=rss&ns_campaign=1490&ito=1490')
articleText = trafilatura.extract(downloaded)

NLTK Experimentation - Unused Currently.

In [ ]:

# # Download the NLTK Punkt tokenizer data if not already downloaded
# nltk.download('punkt')
# 
# from nltk.tokenize import sent_tokenize
# import nltk
# 
# # Download the NLTK Punkt tokenizer data if not already downloaded
# nltk.download('punkt')
# 
# from nltk.tokenize import sent_tokenize
# 
# # Split the text into sentences
# sentences = sent_tokenize(articleText)
# 
# # Print the sentences
# for sentence in sentences:
#     print("sentence:" + sentence)


In [ ]:
import spacy
nlp = spacy.load("en_coreference_web_trf")
doc = nlp("The cats were startled by the dog as it growled at them.") 
print(doc.spans)

In [ ]:
import spacy
doc = nlp("This is a sentence.")
coref = nlp.add_pipe("experimental_coref")
# This usually happens under the hood
processed = coref(doc)

In [ ]:
import torch

In [ ]:
# Check Metal Performance Shaders (MPS) backend for GPU training acceleration.
if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    x = torch.ones(1, device=mps_device)
    print (x)
else:
    print ("MPS device not found.")

In [ ]:
from fastcoref import FCoref

model = FCoref(device='mps')

preds = model.predict(
   texts=['We are so happy to see you using our coref package. This package is very fast!']
)

preds[0].get_clusters(as_strings=False)


print(preds[0].get_clusters())


preds[0].get_logit(
   span_i=(33, 50), span_j=(52, 64)
)


Playing around with extracting coreference clusters using fastcoref directly
"Processing can be applied to a collection of texts of any length in a batched and parallel fashion"
https://pypi.org/project/fastcoref/

In [133]:
from fastcoref import FCoref

model = FCoref(device='mps')

preds = model.predict(
   texts=[articleText]
)

clustersText = preds[0].get_clusters()
clustersPositions = preds[0].get_clusters(as_strings=False)

print(clustersText)

combined_clusters = [(text, positions, len(text)) for text, positions in zip(clustersText, clustersPositions)]

sorted_combined_clusters = sorted(combined_clusters, key=lambda x: x[2], reverse=True)

for i, (clusterText, clusterPos, cluster_text_count) in enumerate(sorted_combined_clusters, 1):
    print(f"Cluster {i} : {clusterText}")
    print(f"Cluster Text Count: {cluster_text_count}")
    print("Positions:", clusterPos)


10/31/2023 13:06:40 - INFO - 	 missing_keys: []
10/31/2023 13:06:40 - INFO - 	 unexpected_keys: []
10/31/2023 13:06:40 - INFO - 	 mismatched_keys: []
10/31/2023 13:06:40 - INFO - 	 error_msgs: []
10/31/2023 13:06:40 - INFO - 	 Model Parameters: 90.5M, Transformer: 82.1M, Coref head: 8.4M
10/31/2023 13:06:40 - INFO - 	 Tokenize 1 inputs...


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

10/31/2023 13:06:40 - INFO - 	 ***** Running Inference on 1 texts *****


Inference:   0%|          | 0/1 [00:00<?, ?it/s]

[['Sadiq Khan', 'his', "Sadiq Khan's", 'London mayor', "Khan's", "the capital's mayor Sadiq Khan", 'Mr Khan', "London mayor Sadiq Khan's", 'Mr Khan', "Sir Keir Starmer's mayor", 'Sadiq Khan', "Mr Khan's", 'Mr Khan', 'I', 'I', 'Mr Khan', 'Sadiq', 'He', 'he', 'his', 'he', 'your mayor', 'the Labour mayor', 'his', "Mr Khan's", "the mayor's", 'his', 'Mr Khan', 'Mr Khan', 'the mayor', "the mayor's"], ['Rishi Sunak', 'Rishi Sunak', 'Rishi Sunak', 'Rishi Sunak (pictured)'], ['Keir Starmer', 'Starmer', 'Keir Starmer', 'him', 'he', 'Sir Keir', 'He', 'the Labour leader', 'Keir Starmer (left)', 'him', 'the Labour leader', "Sir Keir Starmer's", 'Starmer', 'Sir Keir', 'the Labour leader', 'Keir', 'he', 'Sir Keir', 'Sir Keir'], ["Sadiq Khan's Ulez", 'ULEZ', 'ULEZ'], ['London', "London's", "the capital's", 'London', "London's", 'London', 'London', "London's", 'London', "London's", 'London'], ['the levy', 'the Ulez levy', 'the £12.50 daily levy for older, more polluting vehicles', 'it', 'the £12.50 dai

In [ ]:
from fastcoref import LingMessCoref

model = LingMessCoref(device='cpu')

preds = model.predict(
   texts=[articleText]
)

print(preds[0].get_clusters()) 

In [ ]:
# TODO: look into spacy pipelines, spans, etc

In [ ]:
from fastcoref import spacy_component
# Cell not required anymore

import spacy

text = articleText 

nlp = spacy.load("en_core_web_sm")
nlp.add_pipe("fastcoref")

doc = nlp(text)
doc._.coref_clusters


But to my current knowledge: The spaCy library itself does not natively support processing an array of text documents in a single call in parallel.
So consider applying fastCoref directly then using spaCy or alternative to extract sentences

https://medium.com/quantrium-tech/top-3-packages-for-named-entity-recognition-e9e14f6f0a2a
    
And so decided to play around with flair

In [ ]:
# !pip install flair

In [55]:
from flair.data import Sentence
from flair.models import SequenceTagger

# List of entities
entity_list = [
    'Sadiq Khan', 'his', "Sadiq Khan's", 'London mayor', "Khan's",
    "the capital's mayor Sadiq Khan", 'Mr Khan', "London mayor Sadiq Khan's",
    'Mr Khan', "Sir Keir Starmer's mayor", 'Sadiq Khan', "Mr Khan's", 'Mr Khan',
    'I', 'I', 'Mr Khan', 'Sadiq', 'He', 'he', 'his', 'he', 'your mayor',
    'the Labour mayor', 'his', "Mr Khan's", "the mayor's", 'his', 'Mr Khan',
    'Mr Khan', 'the mayor', "the mayor's"
]

# Create a sentence with the entities
sentence = Sentence(" ".join(entity_list))

# Load Flair's NER model (e.g., English NER model)
tagger = SequenceTagger.load("ner")

# Predict NER tags for the sentence
tagger.predict(sentence)

# Extract NER tags and entities
ner_tags = sentence.to_dict(tag_type="ner")["entities"]

# Print the NER tags and entities
for entity in ner_tags:
    print(f"Entity: {entity['text']}, Label: {entity['type']}")



pytorch_model.bin:   0%|          | 0.00/432M [00:00<?, ?B/s]

2023-10-30 19:00:16,728 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>


KeyError: 'type'

In [62]:
# Decided to explore more than just org and person detections - comment out this cell:
# import spacy
# from flair.data import Sentence
# from flair.models import SequenceTagger
# 
# NER = spacy.load("en_core_web_sm")
# 
# article_text = [articleText][0]
# 
# article = NER(article_text)
# org_entities = {'ORG': []}
# person_entities = {'PERSON': []}
# 
# for word in article.ents:
#     label = word.label_
#     if label in org_entities:
#         org_entities[label].append([[word.text, word.start_char, word.end_char], label])
#     if label in person_entities:
#         person_entities[label].append([[word.text, word.start_char, word.end_char], label])
# 
# # Print the ORG entities
# print("ORG Entities:")
# for label, entities in org_entities.items():
#     for entity in entities:
#         entity_text, start_pos, end_pos = entity[0]
#         label = entity[1]
#         print(f"Entity: {entity_text}, Position: [{start_pos}, {end_pos}], Label: {label}")
# 
# # Print the PERSON entities
# print("PERSON Entities:")
# for label, entities in person_entities.items():
#     for entity in entities:
#         entity_text, start_pos, end_pos = entity[0]
#         label = entity[1]
#         print(f"Entity: {entity_text}, Position: [{start_pos}, {end_pos}], Label: {label}")

ORG Entities:
Entity: Ulez, Position: [29, 33], Label: ORG
Entity: Ulez, Position: [200, 204], Label: ORG
Entity: Starmer, Position: [253, 260], Label: ORG
Entity: ULEZ, Position: [337, 341], Label: ORG
Entity: High Court, Position: [367, 377], Label: ORG
Entity: Ulez, Position: [616, 620], Label: ORG
Entity: The High Court, Position: [627, 641], Label: ORG
Entity: Ulez, Position: [676, 680], Label: ORG
Entity: Labour, Position: [984, 990], Label: ORG
Entity: Labour, Position: [1338, 1344], Label: ORG
Entity: Ulez, Position: [1573, 1577], Label: ORG
Entity: ULEZ, Position: [1639, 1643], Label: ORG
Entity: High Court, Position: [1802, 1812], Label: ORG
Entity: RAC, Position: [2212, 2215], Label: ORG
Entity: Driver, Position: [2413, 2419], Label: ORG
Entity: Vehicle Licensing Agency, Position: [2424, 2448], Label: ORG
Entity: Ulez, Position: [2627, 2631], Label: ORG
Entity: the High Court, Position: [2805, 2819], Label: ORG
Entity: Ulez, Position: [2967, 2971], Label: ORG
Entity: Labour,

In [137]:
import spacy
from functools import reduce

NER = spacy.load("en_core_web_sm")

article_text = [articleText][0]

article = NER(article_text)

num_sentences = len(list(article.sents))

threshold = math.floor(num_sentences * 0.2)

# Print the results
print(f"Number of Sentences: {num_sentences}")
print(f"Coreference Threshold: {threshold}")


def merge_positions(entities, word):
    entity_key = word.text + word.label_
    if entity_key in entities:
        entities[entity_key][1].append([word.start_char, word.end_char])
    else:
        entities[entity_key] = [word.text, [[word.start_char, word.end_char]], word.label_]
    return entities

org_entities = reduce(merge_positions, filter(lambda word: word.label_ == 'ORG', article.ents), {})
person_entities = reduce(merge_positions, filter(lambda word: word.label_ == 'PERSON', article.ents), {})

# Print the merged ORG entities
print("Merged ORG Entities:")
for entity_key, entity_info in org_entities.items():
    entity_text = entity_info[0]
    positions = entity_info[1]
    label = entity_info[2]
    print(f"Entity: {entity_text}, Positions: {positions}, Label: {label}")

# Print the merged PERSON entities
print("Merged PERSON Entities:")
for entity_key, entity_info in person_entities.items():
    entity_text = entity_info[0]
    positions = entity_info[1]
    label = entity_info[2]
    print(f"Entity: {entity_text}, Positions: {positions}, Label: {label}")


Number of Sentences: 59
Coreference Threshold: 11
Merged ORG Entities:
Entity: Ulez, Positions: [[29, 33], [200, 204], [616, 620], [676, 680], [1573, 1577], [2627, 2631], [2967, 2971], [3342, 3346], [4182, 4186], [5356, 5360], [5650, 5654], [5925, 5929], [7350, 7354]], Label: ORG
Entity: Starmer, Positions: [[253, 260]], Label: ORG
Entity: ULEZ, Positions: [[337, 341], [1639, 1643]], Label: ORG
Entity: High Court, Positions: [[367, 377], [1802, 1812]], Label: ORG
Entity: The High Court, Positions: [[627, 641], [5739, 5753]], Label: ORG
Entity: Labour, Positions: [[984, 990], [1338, 1344], [3169, 3175], [3295, 3301], [5330, 5336], [6554, 6560]], Label: ORG
Entity: RAC, Positions: [[2212, 2215], [7202, 7205]], Label: ORG
Entity: Driver, Positions: [[2413, 2419]], Label: ORG
Entity: Vehicle Licensing Agency, Positions: [[2424, 2448]], Label: ORG
Entity: the High Court, Positions: [[2805, 2819]], Label: ORG
Entity: Times Radio, Positions: [[3599, 3610]], Label: ORG
Entity: Energy, Position

In [136]:
import spacy
import math
from functools import reduce

NER = spacy.load("en_core_web_sm")

article_text = [articleText][0]

article = NER(article_text)

for ent in article.ents:
    print(f"Entity: {ent.text}, Start: {ent.start_char}, End: {ent.end_char}, Label: {ent.label_}")

Entity: Ulez, Start: 29, End: 33, Label: ORG
Entity: Brits, Start: 54, End: 59, Label: NORP
Entity: Rishi, Start: 62, End: 67, Label: PERSON
Entity: Sadiq Khan, Start: 81, End: 91, Label: PERSON
Entity: Keir Starmer, Start: 141, End: 153, Label: PERSON
Entity: Sadiq Khan's, Start: 187, End: 199, Label: FAC
Entity: Ulez, Start: 200, End: 204, Label: ORG
Entity: Tory, Start: 215, End: 219, Label: PERSON
Entity: Starmer, Start: 253, End: 260, Label: ORG
Entity: London, Start: 268, End: 274, Label: GPE
Entity: ULEZ, Start: 337, End: 341, Label: ORG
Entity: Khan, Start: 360, End: 364, Label: PERSON
Entity: High Court, Start: 367, End: 377, Label: ORG
Entity: Keir Starmer, Start: 382, End: 394, Label: PERSON
Entity: last night, Start: 413, End: 423, Label: TIME
Entity: London, Start: 465, End: 471, Label: GPE
Entity: Sadiq Khan, Start: 569, End: 579, Label: PERSON
Entity: Ulez, Start: 616, End: 620, Label: ORG
Entity: The High Court, Start: 627, End: 641, Label: ORG
Entity: yesterday, Start:

Possible NER outcomes: CARDINAL, DATE, EVENT, FAC, GPE, LANGUAGE, LAW, LOC, MONEY, NORP, ORDINAL, ORG, PERCENT, PERSON, PRODUCT, QUANTITY, TIME, WORK_OF_ART
https://huggingface.co/spacy/en_core_web_sm

In [64]:
def merge_positions(entities, word):
    entity_key = (word.text + word.label_).lower()
    if entity_key in entities:
        entities[entity_key][1].append([word.start_char, word.end_char])
    else:
        entities[entity_key] = [word.text, [[word.start_char, word.end_char]], word.label_]
    return entities

org_entities = reduce(merge_positions, filter(lambda word: word.label_ == 'ORG', article.ents), {})
person_entities = reduce(merge_positions, filter(lambda word: word.label_ == 'PERSON', article.ents), {})


print("Merged ORG Entities:")
for entity_key, entity_info in org_entities.items():
    entity_text = entity_info[0]
    positions = entity_info[1]
    label = entity_info[2]
    print(f"Entity: {entity_text}, Positions: {positions}, Label: {label}")


print("Merged PERSON Entities:")
for entity_key, entity_info in person_entities.items():
    entity_text = entity_info[0]
    positions = entity_info[1]
    label = entity_info[2]
    print(f"Entity: {entity_text}, Positions: {positions}, Label: {label}")
    


Merged ORG Entities:
Entity: Ulez, Positions: [[29, 33], [200, 204], [337, 341], [616, 620], [676, 680], [1573, 1577], [1639, 1643], [2627, 2631], [2967, 2971], [3342, 3346], [4182, 4186], [5356, 5360], [5650, 5654], [5925, 5929], [7350, 7354]], Label: ORG
Entity: Starmer, Positions: [[253, 260]], Label: ORG
Entity: High Court, Positions: [[367, 377], [1802, 1812]], Label: ORG
Entity: The High Court, Positions: [[627, 641], [2805, 2819], [5739, 5753]], Label: ORG
Entity: Labour, Positions: [[984, 990], [1338, 1344], [3169, 3175], [3295, 3301], [5330, 5336], [6554, 6560]], Label: ORG
Entity: RAC, Positions: [[2212, 2215], [7202, 7205]], Label: ORG
Entity: Driver, Positions: [[2413, 2419]], Label: ORG
Entity: Vehicle Licensing Agency, Positions: [[2424, 2448]], Label: ORG
Entity: Times Radio, Positions: [[3599, 3610]], Label: ORG
Entity: Energy, Positions: [[4010, 4016]], Label: ORG
Entity: Londoners, Positions: [[5404, 5413]], Label: ORG
Entity: Surrey County Council, Positions: [[5859,

NER could be confused and additional mentions could be used to determine "correct" category

In [150]:
import spacy
from functools import reduce

NER = spacy.load("en_core_web_sm")

article_text = [articleText][0]

article = NER(article_text)

num_sentences = len(list(article.sents))

threshold = math.floor(num_sentences * 0.2)

# Print the results
print(f"Number of Sentences: {num_sentences}")
print(f"Coreference Threshold: {threshold}")




entity_types = ["CARDINAL", "DATE", "EVENT", "FAC", "GPE", "LANGUAGE", "LAW", "LOC", "MONEY", "NORP", "ORDINAL", "ORG", "PERCENT", "PERSON", "PRODUCT", "QUANTITY", "TIME", "WORK_OF_ART"]

entity_type_to_entities = {
    entity_type: [
        [
            entity_text,
            positions,
            label,
            len(positions)  # Count of positions
        ] for entity_text, positions, label in reduce(
            merge_positions,
            filter(lambda word: word.label_ == entity_type, article.ents),
            {}
        ).values()
    ] for entity_type in entity_types
}

# print(entity_type_to_entities)

for entity_type, entities in entity_type_to_entities.items():
    print(f"Entity Type: {entity_type}")
    for entity in entities:
        entity_text, positions, label, num_positions = entity
        print(f"Entity: {entity_text}")
        print(f"Positions: {positions}")
        print(f"Label: {label}")
        print(f"Number of Positions: {num_positions}")
        print()
        
people_entities = {entity_type: entity_info for entity_type, entity_info in entity_type_to_entities.items() if entity_type == 'PERSON'}


Number of Sentences: 59
Coreference Threshold: 11
Entity Type: CARDINAL
Entity: nearly 700,000
Positions: [[2131, 2145]]
Label: CARDINAL
Number of Positions: 1

Entity: More than 1.5million
Positions: [[2299, 2319]]
Label: CARDINAL
Number of Positions: 1

Entity: nine
Positions: [[2520, 2524]]
Label: CARDINAL
Number of Positions: 1

Entity: ten
Positions: [[2532, 2535]]
Label: CARDINAL
Number of Positions: 1

Entity: five
Positions: [[6309, 6313]]
Label: CARDINAL
Number of Positions: 1

Entity: Seven
Positions: [[6392, 6397]]
Label: CARDINAL
Number of Positions: 1

Entity Type: DATE
Entity: yesterday
Positions: [[642, 651], [900, 909], [2483, 2492]]
Label: DATE
Number of Positions: 3

Entity: daily
Positions: [[1041, 1046], [1395, 1400], [1768, 1773], [2181, 2186], [4268, 4273], [4628, 4633]]
Label: DATE
Number of Positions: 6

Entity: August 29
Positions: [[1129, 1138]]
Label: DATE
Number of Positions: 1

Entity: 29 August
Positions: [[1786, 1795]]
Label: DATE
Number of Positions: 1



In [151]:

for entity, details in people_entities.items():
    print(entity)
    print("Details:", details)
    print()

PERSON
Details: [['Rishi', [[62, 67], [1269, 1274]], 'PERSON', 2], ['Sadiq Khan', [[81, 91], [569, 579], [1243, 1253], [1735, 1745]], 'PERSON', 4], ['Keir Starmer', [[141, 153], [382, 394], [1140, 1152], [1483, 1495]], 'PERSON', 4], ['Tory', [[215, 219]], 'PERSON', 1], ['Khan', [[360, 364], [1009, 1013], [1363, 1367], [2102, 2106], [2506, 2510], [3396, 3400], [5918, 5922], [6345, 6349], [7106, 7110]], 'PERSON', 9], ['Keir', [[769, 773], [3079, 3083], [3613, 3617], [4090, 4094], [5283, 5287]], 'PERSON', 5], ['Rishi Sunak', [[926, 937]], 'PERSON', 1], ['Starmer', [[1688, 1695]], 'PERSON', 1], ["Boris Johnson's", [[3034, 3049]], 'PERSON', 1], ['Rachel Reeves', [[3466, 3479], [3696, 3709]], 'PERSON', 2], ['Wes Streeting', [[3579, 3592]], 'PERSON', 1], ['Grant Shapps', [[4027, 4039]], 'PERSON', 1], ['Ulez', [[4200, 4204]], 'PERSON', 1], ['Trader', [[4811, 4817]], 'PERSON', 1], ['Ian Plummer', [[4996, 5007]], 'PERSON', 1], ['Lucy Frazer', [[5259, 5270]], 'PERSON', 1], ['Mark Harper', [[5436,

In [ ]:
# collated_entities = {}
# 
# for entity_type, entities in entity_type_to_entities.items():
#     for entity_key, entity_info in entities.items():
#         entity_text = entity_info[0]
#         positions = entity_info[1]
#         label = entity_info[2]
# 
#         # Check if the entity text is already in the dictionary
#         if entity_text in collated_entities:
#             # If it is, update the label-specific occurrences
#             if label in collated_entities[entity_text]:
#                 collated_entities[entity_text][label] += len(positions)
#             else:
#                 collated_entities[entity_text][label] = len(positions)
#         else:
#             # If it's not, create a new entry for the entity text
#             collated_entities[entity_text] = {label: len(positions)}
# 
# 
# for entity_text, label_occurrences in collated_entities.items():
#     print(f"Entity Text: {entity_text}")
#     for label, occurs in label_occurrences.items():
#         print(f"Label: {label}, Occurrences: {occurs}")


In [116]:

collated_entities = {}


for entity_type, entities in entity_type_to_entities.items():
    for entity_key, entity_info in entities.items():
        entity_text = entity_info[0]
        positions = entity_info[1]
        label = entity_info[2]

        # Check if the entity text is already in the collated dictionary
        if entity_text in collated_entities:
            # If it is, update the label-specific occurrences and positions
            if label in collated_entities[entity_text]:
                collated_entities[entity_text][label]['occurrences'] += len(positions)
                collated_entities[entity_text][label]['positions'].extend(positions)
            else:
                collated_entities[entity_text][label] = {'occurrences': len(positions), 'positions': positions}
        else:
            # If it's not, create a new entry for the entity text
            collated_entities[entity_text] = {label: {'occurrences': len(positions), 'positions': positions}}


entities_with_multiple_labels = {}

# for entity_text, label_occurrences in collated_entities.items():
#     output = [f"Entity Text: {entity_text}"]
#     for label, data in label_occurrences.items():
#         output.append(f"Label: {label}, Occurrences: {data['occurrences']}, Positions: {data['positions']}")
#     print(" | ".join(output))
    

# Print results where there are duplicate / in dispute NER labels:

# for entity_text, label_occurrences in collated_entities.items():
#     # Check if there are multiple labels associated with the entity text
#     if len(label_occurrences) > 1:
#         output = [f"Entity Text: {entity_text}"]
#         for label, data in label_occurrences.items():
#              output.append(f"Label: {label}, Occurrences: {data['occurrences']}, Positions: {data['positions']}")
#              # output.append(f"Label: {label}, Occurrences: {data['occurrences']}")
#         entities_with_multiple_labels[entity_text] = " | ".join(output)
#         print(" | ".join(output))
#         
# 
# 
# for entity_text, data in entities_with_multiple_labels.items():
#     print(f" {data}")

# Store results where there are duplicate / in dispute NER labels:
for entity_text, label_occurrences in collated_entities.items():
    if len(label_occurrences) > 1:
        entities_with_multiple_labels[entity_text] = []
        for label, data in label_occurrences.items():
            entity_info = {
                "Label": label,
                "Occurrences": data['occurrences'],
                "Positions": data['positions']
            }
            entities_with_multiple_labels[entity_text].append(entity_info)

# Print the stored alike entity text values that have multiple NER labels:
for entity_text, label_data in entities_with_multiple_labels.items():
    print("---------------------------")
    print(f"Entity Text: {entity_text}")
    for label_info in label_data:
        print(f"Label: {label_info['Label']}, Occurrences: {label_info['Occurrences']}, Positions: {label_info['Positions']}")


---------------------------
Entity Text: Ulez
Label: GPE, Occurrences: 4, Positions: [[2388, 2392], [3533, 3537], [4657, 4661], [6034, 6038]]
Label: ORG, Occurrences: 15, Positions: [[29, 33], [200, 204], [337, 341], [616, 620], [676, 680], [1573, 1577], [1639, 1643], [2627, 2631], [2967, 2971], [3342, 3346], [4182, 4186], [5356, 5360], [5650, 5654], [5925, 5929], [7350, 7354]]
Label: PERSON, Occurrences: 1, Positions: [[4200, 4204]]
---------------------------
Entity Text: Greater London
Label: GPE, Occurrences: 1, Positions: [[5671, 5685]]
Label: LOC, Occurrences: 1, Positions: [[2158, 2172]]
---------------------------
Entity Text: Tory
Label: NORP, Occurrences: 1, Positions: [[3207, 3211]]
Label: PERSON, Occurrences: 1, Positions: [[215, 219]]
---------------------------
Entity Text: Starmer
Label: ORG, Occurrences: 1, Positions: [[253, 260]]
Label: PERSON, Occurrences: 1, Positions: [[1688, 1695]]
---------------------------
Entity Text: Londoners
Label: ORG, Occurrences: 1, Posit

Appears using the majority case wouldn't always work. 

Let us park these inconclusive NER classifications here ^^^. After we have connected the NER to the coreference resolution output. We can 

In [118]:
nlp = spacy.load("en_core_web_sm")

coreference_matrix = [
    'Sadiq Khan', 'his', "Sadiq Khan's", 'London mayor', "Khan's", "the capital's mayor Sadiq Khan",
    'Mr Khan', "London mayor Sadiq Khan's", 'Mr Khan', "Sir Keir Starmer's mayor", 'Sadiq Khan',
    "Mr Khan's", 'Mr Khan', 'I', 'I', 'Mr Khan', 'Sadiq', 'He', 'he', 'his', 'he', 'your mayor',
    'the Labour mayor', 'his', "Mr Khan's", "the mayor's", 'his', 'Mr Khan', 'Mr Khan', 'the mayor', "the mayor's"
]

split_matrix = [element.split() for element in coreference_matrix]

nouns = []
for words in split_matrix:
    for word in words:
        doc = nlp(word)
        for token in doc:
            if token.pos_ == "NOUN":
                nouns.append(token.text)

print(nouns)

['Sadiq', 'Sadiq', 'mayor', 'mayor', 'Sadiq', 'mayor', 'Sadiq', 'Sir', 'Keir', 'mayor', 'Sadiq', 'Sadiq', 'mayor', 'Labour', 'mayor', 'mayor', 'mayor', 'mayor']


Seems flawed approach as Keir has been brought out from the phrase "Sir Keir Starmer's mayor" even when we are referring to Sadiq Khan - additionally Khan was discarded too.

Discard a cluster c in a document d if |Mc| ≤ 0.2|Sd|, where |...| is the number of mentions of a cluster (Mc) and sentences in a document (Sd) (NEWS-MTSC approach)

In [126]:
for cluster in clustersText:
    print("-----------------------------")
    most_specific_mention = None
   
    text = " ".join(cluster)
    doc = nlp(text)

    for ent in doc.ents:
        if ent.label_ == "PERSON":
            most_specific_mention = most_specific_mention or ent.text

    if most_specific_mention:
        print(cluster)
        print("Most Specific PERSON Mention:", most_specific_mention)
    else:
        print(cluster)
        print("No specific PERSON name found in the cluster.")

-----------------------------
['Sadiq Khan', 'his', "Sadiq Khan's", 'London mayor', "Khan's", "the capital's mayor Sadiq Khan", 'Mr Khan', "London mayor Sadiq Khan's", 'Mr Khan', "Sir Keir Starmer's mayor", 'Sadiq Khan', "Mr Khan's", 'Mr Khan', 'I', 'I', 'Mr Khan', 'Sadiq', 'He', 'he', 'his', 'he', 'your mayor', 'the Labour mayor', 'his', "Mr Khan's", "the mayor's", 'his', 'Mr Khan', 'Mr Khan', 'the mayor', "the mayor's"]
Most Specific PERSON Mention: Khan
-----------------------------
['Rishi Sunak', 'Rishi Sunak', 'Rishi Sunak', 'Rishi Sunak (pictured)']
Most Specific PERSON Mention: Rishi
-----------------------------
['Keir Starmer', 'Starmer', 'Keir Starmer', 'him', 'he', 'Sir Keir', 'He', 'the Labour leader', 'Keir Starmer (left)', 'him', 'the Labour leader', "Sir Keir Starmer's", 'Starmer', 'Sir Keir', 'the Labour leader', 'Keir', 'he', 'Sir Keir', 'Sir Keir']
Most Specific PERSON Mention: Keir Starmer Starmer Keir Starmer
-----------------------------
["Sadiq Khan's Ulez", 'ULE

In [77]:
# TODO: Resolve conflict code above to identify prevalence of NER unsure.
# TODO: Match co-reference resolution outcomes with NER identification to increase positions code.
# TODO: WIKIPEDIA Resolution? (Nice to have? for summaries?)
# TODO: Extract sentences for all identified mentions
# TODO: Apply NewsSentiment with correct start/end positions and formatting resolution